In [2]:
from datetime import datetime as dt

BBOXES = {
    'central': [29.242740, -0.807347, 29.629208, -0.591585],
    'south':  [29.296298, -1.582263, 29.769293, -1.197520],
}

JOBS = [
    (dt(2020, 1, 1),  dt(2020, 3, 1),  'south', 40),
    (dt(2020, 4, 1),  dt(2020, 6, 1),  'south', 30),
    (dt(2020, 7, 1),  dt(2020, 9, 1),  'south', 15),
    (dt(2020, 10, 1), dt(2020, 12, 1), 'south', 20),
    
    (dt(2021, 1, 1),  dt(2021, 3, 1), 'south',  30),
    (dt(2021, 4, 1),  dt(2021, 6, 1), 'south',  30),
    (dt(2021, 7, 1),  dt(2021, 9, 1), 'south',  15),
    (dt(2021, 10, 1), dt(2021, 12, 1), 'south', 20),
    
    (dt(2022, 1, 1),  dt(2022, 3, 1), 'south',  20),
    (dt(2022, 4, 1),  dt(2022, 6, 1), 'south',  30),
    (dt(2022, 7, 1),  dt(2022, 9, 1), 'south',   5),
    (dt(2022, 10, 1), dt(2022, 12, 1), 'south', 30),
    
    (dt(2020, 1, 1),  dt(2020, 3, 1),  'central', 20),
    (dt(2020, 4, 1),  dt(2020, 6, 1),  'central', 20),
    (dt(2020, 7, 1),  dt(2020, 9, 1),  'central', 20),
    (dt(2020, 10, 1), dt(2020, 12, 1), 'central', 20),

    (dt(2021, 1, 1),  dt(2021, 3, 1), 'central',  20),
    (dt(2021, 4, 1),  dt(2021, 6, 1), 'central',  30),
    (dt(2021, 7, 1),  dt(2021, 9, 1), 'central',  20),
    (dt(2021, 10, 1), dt(2021, 12, 1), 'central', 20),

    (dt(2022, 1, 1),  dt(2022, 3, 1), 'central',  30),
    (dt(2022, 4, 1),  dt(2022, 6, 1), 'central',  20),
    (dt(2022, 7, 1),  dt(2022, 9, 1), 'central',  15),
    (dt(2022, 10, 1), dt(2022, 12, 1), 'central', 30),
]


## Initialize collections 📂

Try to make sure grid square has at least 2-3 images per 

In [3]:
%run _downloads.ipynb
%run _imagery.ipynb


job_params = []
for job in JOBS:
    start_date, end_date, region, max_cloud_cover = job[0], job[1], job[2], job[3]
    bbox = BBOXES[region]

    sub_dir =  start_date.strftime("%Y%m%d") + "_" + end_date.strftime("%Y%m%d") + "_" + region
    print(sub_dir)
    
    collection = get_collection(start_date, end_date, bbox, f'./data/{sub_dir}/s2_collection.json', max_cloud_cover=max_cloud_cover)
    
    job_params.append({
        'collection': collection,
        'sub_dir': sub_dir,
        'bbox': bbox
    })
    

20200101_20200301_south
	./data/20200101_20200301_south/s2_collection.json: 6 items found
	{'QU': 3, 'RU': 3}
20200401_20200601_south
	./data/20200401_20200601_south/s2_collection.json: 4 items found
	{'QU': 2, 'RU': 2}
20200701_20200901_south
	./data/20200701_20200901_south/s2_collection.json: 7 items found
	{'QU': 2, 'RU': 4}
20201001_20201201_south
	./data/20201001_20201201_south/s2_collection.json: 6 items found
	{'QU': 3, 'RU': 3}
20210101_20210301_south
	./data/20210101_20210301_south/s2_collection.json: 4 items found
	{'QU': 2, 'RU': 2}
20210401_20210601_south
	./data/20210401_20210601_south/s2_collection.json: 6 items found
	{'RU': 3, 'QU': 3}
20210701_20210901_south
	./data/20210701_20210901_south/s2_collection.json: 8 items found
	{'QU': 2, 'RU': 4}
20211001_20211201_south
	./data/20211001_20211201_south/s2_collection.json: 5 items found
	{'QU': 3, 'RU': 2}
20220101_20220301_south
	./data/20220101_20220301_south/s2_collection.json: 8 items found
	{'QU': 2, 'RU': 4}
20220401_2

In [18]:
%run _downloads.ipynb
%run _imagery.ipynb


for params in job_params:
    bbox = params['bbox']
    sub_dir = params['sub_dir']
    collection = params['collection']
    
    print(sub_dir)
    
    composites = get_processed_composites(collection, bbox, f'./data/{sub_dir}', overwriteDownload=False, overwriteProcessing=True)

    # prepare RGB-TIFs for labeling and map tiles
    
    rgb_band_paths = [composites['B04'], composites['B03'], composites['B02']]

    rgb_uint16_vrt_path = f'./data/{sub_dir}/rgb_uint16.vrt'
    create_vrt(rgb_band_paths, rgb_uint16_vrt_path)

    rgb_byte_vrt_path = f'./data/{sub_dir}/rgb_byte.vrt'
    create_byte_vrt(rgb_uint16_vrt_path, rgb_byte_vrt_path)

    rgb_byte_tif_path = f'./data/{sub_dir}/rgb_byte_cog.tif'
    create_tif(rgb_byte_vrt_path, rgb_byte_tif_path, isCog=True)
    
    # prepare all-TIFs for analysis
    
    all_band_paths = list(composites.values())
    
    all_uint16_vrt_path = f'./data/{sub_dir}/all_uint16.vrt'
    create_vrt(all_band_paths, all_uint16_vrt_path)

    all_uint16_tif_path = f'./data/{sub_dir}/all_uint16.tif'
    create_tif(all_uint16_vrt_path, all_uint16_tif_path, isCog=False)
            
        
# TODO: CHECK THAT OVERWRITE FLAGS WORKING FOR PROCESSING... 

20200101_20200301_south
	downloading... S2B_35MQU_20200217_0_L2A
	downloading... S2B_35MRU_20200217_0_L2A
	downloading... S2A_35MQU_20200212_0_L2A
	downloading... S2A_35MRU_20200212_0_L2A
	downloading... S2B_35MQU_20200207_0_L2A
	downloading... S2B_35MRU_20200207_0_L2A
	masking... S2B_35MQU_20200217_0_L2A
	masking... S2B_35MRU_20200217_0_L2A
	masking... S2A_35MQU_20200212_0_L2A
	masking... S2A_35MRU_20200212_0_L2A
	masking... S2B_35MQU_20200207_0_L2A
	masking... S2B_35MRU_20200207_0_L2A
	stacking... B02
	compositing... B02
	stacking... B03
	compositing... B03
	stacking... B04
	compositing... B04
	stacking... B08
	compositing... B08
20200401_20200601_south
	downloading... S2B_35MQU_20200527_0_L2A
	downloading... S2B_35MRU_20200527_0_L2A
	downloading... S2B_35MQU_20200407_0_L2A
	downloading... S2B_35MRU_20200407_0_L2A
	masking... S2B_35MQU_20200527_0_L2A
	masking... S2B_35MRU_20200527_0_L2A
	masking... S2B_35MQU_20200407_0_L2A
	masking... S2B_35MRU_20200407_0_L2A
	stacking... B02
	compos